In [1]:
cd /home/andrew/side_projects/pysyft-ak/PySyft/

In [2]:
import torch as th
import syft as sy
import copy

In [3]:
sy.create_sandbox(globals())

Setting up Sandbox...
	- Hooking PyTorch
	- Creating Virtual Workers:
		- bob
		- theo
		- jason
		- alice
		- andy
		- jon
	Storing hook and workers as global variables...
	Loading datasets from SciKit Learn...
		- Boston Housing Dataset
		- Diabetes Dataset
		- Breast Cancer Dataset
	- Digits Dataset
		- Iris Dataset
		- Wine Dataset
		- Linnerud Dataset
	Distributing Datasets Amongst Workers...
Done!


In [4]:
def get_boston_target_and_data(worker):
    return (
        worker.search('#boston', '#data')[0],
        worker.search('#boston', '#target')[0]
    )

workers = [alice, bob, theo]
crypto_provider = jon

### Just using dictionaries

In [ ]:
XtX_ptrs, Xty_ptrs = {}, {}

# - each worker computes XtX, Xty locally and shares the result with others
# - each worker maintains a pointer to the shared tensor
for worker in workers:
    X, y = get_boston_target_and_data(worker)
    
    XtX_ptr = X.transpose(0, 1).mm(X)
    Xty_ptr = X.transpose(0, 1).mm(y.reshape(-1, 1))
    
    XtX_ptrs[worker.id] = XtX_ptr.share(*workers, crypto_provider=crypto_provider)
    Xty_ptrs[worker.id] = Xty_ptr.share(*workers, crypto_provider=crypto_provider)

In [31]:
XtX_ptr = sum([ptr.get() for ptr in XtX_ptrs.values()])    
Xty_ptr = sum([ptr.get() for ptr in Xty_ptrs.values()])

In [38]:
XtX_ptr.child.reconstruct()

MultiPointerTensor>{'alice': (Wrapper)>[PointerTensor | me:15935319576 -> alice:4056201775], 'bob': (Wrapper)>[PointerTensor | me:32604966963 -> bob:15935319576], 'theo': (Wrapper)>[PointerTensor | me:42770908680 -> theo:15935319576]}

### Leveraging multipointer tensors

In [53]:
pairs = [get_boston_target_and_data(worker) for worker in workers]
X_ptrs = sy.MultiPointerTensor(children=[p[0] for p in pairs])
y_ptrs = sy.MultiPointerTensor(children=[p[1] for p in pairs])

In [54]:
# the "big data" step; each worker uses its local data 
# to compute its summand to Xt * X and Xt * y
Xt_ptrs = X.transpose(0, 1)
XtX_ptrs = Xt.mm(X)
Xty_ptrs = Xt.mm(y.reshape(-1, 1))

In [55]:
# Now need to securely compute the sums Xt * X and Xt * y. 
# - each worker shares its summand with the others
# - one worker (currently, 'me') gets pointers to the shared summands and adds them up
XtX_shared_ptrs = []
Xty_shared_ptrs = []

for worker in workers:
    XtX_shared_ptrs.append(
        XtX_ptrs.child[worker.id].share(*workers).get()
    )
    
    Xty_shared_ptrs.append(
        Xty_ptrs.child[worker.id].share(*workers).get()
    )
    
XtX_ptr = sum(XtX_shared_ptrs)
Xty_ptr = sum(Xty_shared_ptrs)

In [56]:
# The tough part; need to invert XtX. For now, just pretend that XtX is its own inverse.
beta = XtX_ptr.mm(Xty_ptr)


In [57]:
# reconstruct beta locally on workers
reconstructed = beta.child.reconstruct()

In [64]:
got = reconstructed.child['alice'].get()

In [66]:
type(got)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [42]:
reconstructed

MultiPointerTensor>{'alice': tensor([]), 'bob': tensor([]), 'theo': tensor([])}